# Reglas de asociación con el dataset MovieLens


### Consigna:

* Obtener reglas de asociación entre películas en el dataset movielens (como si fuera recomendación!) 
* Aplicar diferentes métricas de ordenamiento
* Hacer un pequeño informe 



__Primero vemos como está formado el dataset:__

It contains 20000263 ratings and 465564 tag applications across 27278 movies. These data were created by 138493 users between January 09, 1995 and March 31, 2015. This dataset was generated on March 31, 2015, and updated on October 17, 2016 to update links.csv and add genome-* files.

The data are contained in __six files__, 'genome-scores.csv', 'genome-tags.csv', 'links.csv', 'movies.csv', 'ratings.csv' and 'tags.csv'.

__User ids__ are consistent between 'ratings.csv' and 'tags.csv' (i.e., the same id refers to the same user across the two files).

These __movie ids__ are consistent with those used on the MovieLens web site (e.g., id `1` corresponds to the URL <https://movielens.org/movies/1>). Movie ids are consistent between 'ratings.csv', 'tags.csv', 'movies.csv', and 'links.csv' 


### ratings.csv

All __ratings__ are contained in the file 'ratings.csv'. Each line of this file after the header row represents one rating of one movie by one user, and has the following format: userId,movieId,rating,timestamp.

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

### tags.csv

All __tags__ are contained in the file 'tags.csv'. Each line of this file after the header row represents one tag applied to one movie by one user, and has the following format: userId,movieId,tag,timestamp

The lines within this file are ordered first by userId, then, within user, by movieId.

Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user.

### movies.csv

Each line of this file after the header row represents one movie, and has the following format: movieId,title,genres

Movie titles are entered manually or imported from <https://www.themoviedb.org/>, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

Genres: 
* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western
* (no genres listed)

### links.csv

Identifiers that can be used to link to other sources of movie data are contained in the file `links.csv`. Each line of this file after the header row represents one movie, and has the following format: movieId,imdbId,tmdbId

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from apyori import apriori

### Carga y una primera mirada a los datos:

In [102]:
ratings = pd.read_csv('./ml-20m/ratings.csv')

In [103]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [104]:
ratings.shape

(20000263, 4)

In [105]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
userId       int64
movieId      int64
rating       float64
timestamp    int64
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


Se observa que no hay valores nulos y los tipos de datos son los correctos.

In [106]:
tags = pd.read_csv('./ml-20m/tags.csv')

In [107]:
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [108]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
userId       465564 non-null int64
movieId      465564 non-null int64
tag          465548 non-null object
timestamp    465564 non-null int64
dtypes: int64(3), object(1)
memory usage: 14.2+ MB


Se observa que no todas las puntuaciones hechas por usuarios fueron acompañadas por una etiqueta. De todas formas no se va a usar éste archivo para el práctico.

In [22]:
movies = pd.read_csv('./ml-20m/movies.csv')

In [23]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [111]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null int64
title      27278 non-null object
genres     27278 non-null object
dtypes: int64(1), object(2)
memory usage: 639.4+ KB


Se observa si coindice la cantidad de películas puntuadas en ratings con las del dataset movies

In [112]:
listmovies = ratings.movieId.unique().tolist()

print("Hay ", len(listmovies), " películas puntuadas por los usuarios en ratings.")
print("Sus movieId van de ", min(listmovies), " a ", max(listmovies))

Hay  26744  películas puntuadas por los usuarios en ratings.
Sus movieId van de  1  a  131262


En el dataframe movies estan cargadas 27278 películas.

In [113]:
listmovIDs = movies.movieId.unique().tolist()
print("Los MovieIds en movies van de ", min(listmovIDs), " a ", max(listmovIDs))

Los MovieIds en movies van de  1  a  131262


In [114]:
for i in listmovies:
    if not(i in listmovIDs):
        print("Faltan movieIds en el dataset movies.")
        break
    elif i == listmovies[-1]:
        print("Todo ok!")


Todo ok!


### A los fines de éste práctico nos quedamos con con los dataframes: ratings y movies.

### Definimos:
* El conjunto de ítems I: son todas las películas del dataset.
* Un ítem es: una película vista por un usuario.
* Una transacción t es: el conjunto de películas que vió un usuario.


Para empezar, sólo me interesa recomendar las películas que fueron puntuadas "positivamente" por los usuarios que las vieron. Con ésto mejoro mi sistema de recomendación y además se consigue filtrar el dataset para pasar menos transacciones al algoritmo apriori cuando buscamos las reglas de asociación.

In [115]:
ratings = ratings[["userId", "movieId", "rating"]]
ratings.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


Primero agrego una nueva columna con los ratings normalizados por usuarios:

In [118]:
# calculo un zscore por usuario
medias = ratings['rating'].groupby(ratings.userId).mean()
desviaciones = ratings['rating'].groupby(ratings.userId).std()   

In [121]:
ratings['medias'] = ratings['userId'].apply(lambda userid: medias[userid])

In [122]:
ratings['desviaciones'] = ratings['userId'].apply(lambda userid: desviaciones[userid])

In [123]:
ratings['ratingNorm'] = (ratings.rating-ratings.medias)/ratings.desviaciones

In [125]:
ratings.head()

,userId,movieId,rating,medias,desviaciones,ratingNorm
0,1,2,3.5,3.742857,0.382284,-0.635279
1,1,29,3.5,3.742857,0.382284,-0.635279
2,1,32,3.5,3.742857,0.382284,-0.635279
3,1,47,3.5,3.742857,0.382284,-0.635279
4,1,50,3.5,3.742857,0.382284,-0.635279


In [130]:
ratings.to_csv('ratNorm.csv', header=True, index=False)

In [ ]:
# ratingsN = pd.read_csv('./ratNorm.csv')

En principio filtraría el dataframe ratings con los ratingNorm>0 positivos. Pero como quedan 10billones de ratings para el apriori y la ejecución se hace muy lenta, decido filtrar un poco más para obtener las películas que más gustaron al usuario. Mejor aún para la recomendación!

In [186]:
sample = ratings[ratings.ratingNorm > 1]
sample.head()

,userId,movieId,rating,medias,desviaciones,ratingNorm
30,1,1196,4.5,3.742857,0.382284,1.980576
31,1,1198,4.5,3.742857,0.382284,1.980576
131,1,4993,5.0,3.742857,0.382284,3.288503
142,1,5952,5.0,3.742857,0.382284,3.288503
158,1,7153,5.0,3.742857,0.382284,3.288503


In [187]:
sample.shape

(3095998, 6)

Sobre éste dataframe buscamos reglas de asociación con apriori. Para ello acomodamos los datos a listas.

In [188]:
usuarios = sample.userId.unique()

In [189]:
transacciones = []
for idx in usuarios:
    transacciones.append(list(sample[sample.userId==idx].movieId))

Guardamos las transacciones obtenidas.

In [ ]:
import pickle

def guardar_datos(datos):
    with open("transacciones.pkl", "wb") as f:
        pickle.dump(datos, f)


def cargar_datos():
     with open("transacciones.pkl", "rb") as f:
         return pickle.load(f)

guardar_datos(transacciones)

In [3]:
transacc = cargar_datos()

In [4]:
type(transacc)

list

__Variamos los parámetros del apriori en busca de las mejores reglas de asociación:__

* __Soporte:__ Para empezar consideremos que hay 27mil peliculas puntuadas y tenemos 3millones de puntuaciones de usuarios para el algoritmo apriori. Si fuera que todas las películas aparecen la misma cantidad de veces (se sabe que no es así) se obtendría un soporte de 0.003 para cada película. Partimos de éste valor hacia arriba para fijar el soporte. Es decir, dada la gran cantidad de valoraciones de usuarios y la gran variedad de películas no se considera necesario un soporte muy alto.

* __Confianza:__ Para recomendar me interesa conseguir reglas de asociación que tengan una buena confianza, es decir, que la probabilidad de que se vea una película Y dado que se vió otra película X sea considerable. Para empezar considero que un 30% esta bien.

* __Lift:__ Además, es necesario que el lift sea >1 pues esto significa que la probabilidad de que se vean dos películas juntas se debe a una correlación y no al azar. 

In [5]:
#3095998/27278
11300/3095998

0.00364987315883279

In [203]:
association_rules = apriori(transacc, min_support=0.01, min_confidence=0.3, min_lift=2, min_length=2)
association_rules = list(association_rules)

In [204]:
print(len(association_rules))

12619


Se consiguieron 12619 reglas de asociación.

Luego, exigiendo un poco más de la confianza y el lift de las reglas se obtiene:

In [8]:
association_rules2 = apriori(transacc, min_support=0.01, min_confidence=0.5, min_lift=3, min_length=2)
association_rules2 = list(association_rules2)

In [9]:
print(len(association_rules2))

7430


Se acomodan las reglas obtenidas en un dataframe.

In [10]:
reglas = pd.DataFrame(columns=['Regla','Soporte','Confianza','Lift'])

for i,item in enumerate(association_rules2):
    reglas.loc[i] = [item[0],item[1],item[2][0][2],item[2][0][3]]

In [11]:
reglas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7430 entries, 0 to 7429
Data columns (total 4 columns):
Regla        7430 non-null object
Soporte      7430 non-null float64
Confianza    7430 non-null float64
Lift         7430 non-null float64
dtypes: float64(3), object(1)
memory usage: 290.2+ KB


In [12]:
reglas.head()

,Regla,Soporte,Confianza,Lift
0,"(1, 3114)",0.023005,0.573395,5.777556
1,"(50, 555)",0.015450,0.537234,3.092420
2,"(266, 110)",0.011218,0.529908,3.510482
3,"(553, 110)",0.013819,0.542000,3.590590
4,"(1408, 110)",0.010156,0.518888,3.437482


__A continuación se ordenan las reglas en orden descendente de lift para visualizar las que tienen mayor lift.__

In [13]:
reglas.sort_values(by='Lift', ascending=False).head(20)

,Regla,Soporte,Confianza,Lift
26,"(306, 307)",0.010521,0.511148,29.997876
174,"(8665, 5418)",0.012382,0.731793,25.038856
175,"(5418, 54286)",0.010513,0.597008,20.427073
2641,"(745, 1148, 1223)",0.012280,0.918627,20.110322
42,"(745, 1223)",0.013368,0.716629,20.096346
3222,"(1089, 6874, 7438)",0.010164,0.666295,18.720542
6182,"(296, 6874, 2571, 7438)",0.010139,0.665365,18.694414
6191,"(296, 6874, 7438, 2959)",0.012000,0.662289,18.607978
2637,"(720, 745, 1148)",0.011006,0.846959,18.541386
2638,"(1136, 745, 1148)",0.010113,0.845771,18.515386


Una vez que se obtienen las reglas de asociación, la medida más interesante es el lift ya que ella guarda información conjunta del soporte y de la confianza. Dice en qué proporción aumenta la vista de una película dado que antes se vió otra película, o qué películas estan más relacionadas a otra según los intereses del usuario. Ésto es justamente lo que se necesita para un sistema de recomendación. Se observa que hay muchas reglas con un lift alto, éstas tienen un soporte mayor a 0.01 pues como se dijo antes no íba a ser tan estricta en el soporte debido a la gran variedad de películas y proporción aún mucho mayor de puntuaciones de usuario reclutadas.

__Probamos ahora viendo aquellas reglas que tienen mayor confianza.__

In [14]:
reglas.sort_values(by='Confianza', ascending=False).head(20)

,Regla,Soporte,Confianza,Lift
2641,"(745, 1148, 1223)",0.012280,0.918627,20.110322
6987,"(5952, 4993, 7153, 7438)",0.010708,0.916364,9.304226
6990,"(5952, 4993, 7153, 58559)",0.013649,0.916144,9.301994
6988,"(5952, 4993, 7153, 8961)",0.011099,0.915207,9.292480
6989,"(5952, 4993, 33794, 7153)",0.011872,0.913669,9.276867
6986,"(5952, 4993, 7361, 7153)",0.012297,0.903246,9.171037
6985,"(5952, 4993, 6874, 7153)",0.013368,0.900916,9.147384
5602,"(1240, 1291, 260, 1198)",0.010462,0.892029,7.084454
5599,"(1291, 260, 1198, 1214)",0.010377,0.889942,7.067877
1477,"(1210, 1387, 260)",0.011422,0.889477,5.042299


Vemos que hay reglas con una confianza muy alta y un buen valor de lift. La confianza habla de la probabilidad de que aparezcan juntas 2, 3, 4.. películas. Éste tambien puede ser un dato útil para recomendar películas considerando que siempre trabajamos con muestras, aún si usáramos todo el dataset, sería una muestra más grande.

Veamos cuales son las películas de las primeras reglas.

In [36]:
reglas = pd.DataFrame(columns=['Regla','Soporte','Confianza','Lift'])

for i,item in enumerate(association_rules2):
    items = [movies[movies.movieId==x].title for x in item[0]]
    reglas.loc[i] = [items,item[1],item[2][0][2],item[2][0][3]]

In [38]:
reglas.sort_values(by='Lift', ascending=False).head(10)

,Regla,Soporte,Confianza,Lift
26,[[Three Colors: Red (Trois couleurs: Rouge) (1...,0.010521,0.511148,29.997876
174,"[[Bourne Supremacy, The (2004)], [Bourne Ident...",0.012382,0.731793,25.038856
175,"[[Bourne Identity, The (2002)], [Bourne Ultima...",0.010513,0.597008,20.427073
2641,"[[Wallace & Gromit: A Close Shave (1995)], [Wa...",0.012280,0.918627,20.110322
42,"[[Wallace & Gromit: A Close Shave (1995)], [Gr...",0.013368,0.716629,20.096346
3222,"[[Reservoir Dogs (1992)], [Kill Bill: Vol. 1 (...",0.010164,0.666295,18.720542
6182,"[[Pulp Fiction (1994)], [Kill Bill: Vol. 1 (20...",0.010139,0.665365,18.694414
6191,"[[Pulp Fiction (1994)], [Kill Bill: Vol. 1 (20...",0.012000,0.662289,18.607978
2637,[[Wallace & Gromit: The Best of Aardman Animat...,0.011006,0.846959,18.541386
2638,"[[Monty Python and the Holy Grail (1975)], [Wa...",0.010113,0.845771,18.515386


In [39]:
reglas.sort_values(by='Confianza', ascending=False).head(10)

,Regla,Soporte,Confianza,Lift
2641,"[[Wallace & Gromit: A Close Shave (1995)], [Wa...",0.012280,0.918627,20.110322
6987,"[[Lord of the Rings: The Two Towers, The (2002...",0.010708,0.916364,9.304226
6990,"[[Lord of the Rings: The Two Towers, The (2002...",0.013649,0.916144,9.301994
6988,"[[Lord of the Rings: The Two Towers, The (2002...",0.011099,0.915207,9.292480
6989,"[[Lord of the Rings: The Two Towers, The (2002...",0.011872,0.913669,9.276867
6986,"[[Lord of the Rings: The Two Towers, The (2002...",0.012297,0.903246,9.171037
6985,"[[Lord of the Rings: The Two Towers, The (2002...",0.013368,0.900916,9.147384
5602,"[[Terminator, The (1984)], [Indiana Jones and ...",0.010462,0.892029,7.084454
5599,"[[Indiana Jones and the Last Crusade (1989)], ...",0.010377,0.889942,7.067877
1477,[[Star Wars: Episode VI - Return of the Jedi (...,0.011422,0.889477,5.042299
